# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [1]:
// Equivalent of adding dependencies to maven or sbt files
// For example, to add "org.scalaj" %% "scalaj-http" % "2.4.2" 
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

In [2]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

### 1.1 Obtener el texto del RSS Feed

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

In [3]:
class Parser() {
    // Get the xml content
    def openFile(url: String): String = {
        try {
            var response = Http(url)
          .timeout(connTimeoutMs = 2000, readTimeoutMs = 5000)
          .asString
            response.body
        } catch {
            case e => println("Error in http response")
            "<rss></rss>"
        }
        
    }
    // Extract text 
    def processText(xmlString: String): collection.immutable.Seq[String] = {
        // convert the `String` to a `scala.xml.Elem`
        val xml = XML.loadString(xmlString)
        // Extract text from title and description
        (xml \\ "item").map { item =>
            ((item \ "title").text ++ " " ++ (item \ "description").text)
        }
    }
    
}

defined class Parser

In [4]:
val parser = new Parser

parser: Parser = ammonite.$sess.cmd2$Helper$Parser@172dda9d

In [5]:
val url = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
val xmlString = parser.openFile(url)
val rssText = parser.processText(xmlString)

url: String = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
xmlString: String = """<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:media="http://search.yahoo.com/mrss/" xmlns:sy="http://purl.org/rss/1.0/modules/syndication/"><channel><title>Chicago Tribune</title><link>https://www.chicagotribune.com</link><language>en-US</language><copyright>© 2021 Chicago Tribune</copyright><atom:link href="https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:%5Bnow-2d+TO+now%5D&amp;sort=display_date:desc" rel="self" type="application/rss+xml"/><description>Chicago Tribune News Feed</description><lastBuildDate>Sun, 23 May 2021 02:10:38 +0000</lastBuildDate><ttl>1</ttl><sy:updatePeriod>hourly</sy:updatePeriod><sy:updateFrequency>1</sy:updateFrequency><i

## 2. Detectar las entidades nombradas

### 2.1 Crear el modelo

El **modelo** es sólo la función `getNEs`, que recibe una lista de textos.
Para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

Este código lista los signos de puntuación y algunas palabras comunes del inglés que se van a sacar del texto.

In [6]:
val STOPWORDS = Seq (
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
    "their", "theirs", "themselves", "what", "which", "who", "whom",
    "this", "that", "these", "those", "am", "is", "are", "was", "were",
    "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or",
    "because", "as", "until", "while", "of", "at", "by", "for", "with",
    "about", "against", "between", "into", "through", "during", "before",
    "after", "above", "below", "to", "from", "up", "down", "in", "out",
    "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each",
    "few", "more", "most", "other", "some", "such", "no", "nor", "not",
    "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
    "will", "just", "don", "should", "now", "on",
    // Contractions without '
    "im", "ive", "id", "Youre", "youd", "youve",
    "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
    "theyre", "theyd", "theyve",
    "shouldnt", "couldnt", "musnt", "cant", "wont",
    // Common uppercase words
    "hi", "hello"
)

STOPWORDS: Seq[String] = List(
  "i",
  "me",
  "my",
  "myself",
  "we",
  "our",
  "ours",
  "ourselves",
  "you",
  "yours",
  "yourself",
  "yourselves",
  "he",
  "him",
  "his",
  "himself",
  "she",
  "her",
  "hers",
  "herself",
  "it",
  "its",
  "itself",
  "they",
  "them",
  "your",
  "their",
  "theirs",
  "themselves",
  "what",
  "which",
  "who",
  "whom",
  "this",
  "that",
  "these",
  "those",
  "am",
...

In [7]:
class NERModel(STOPWORDS: Seq[String]) {
    
    val punctuationSymbols = ".,()!?;:'`´\n"
    val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")
    
    // Extract Named Entitis
    def getNEsSingle(text: String): Seq[String] = {
      text.replaceAll(punctuationRegex, "").split(" ")
        .filter { word:String => word.length > 1 &&
                  Character.isUpperCase(word.charAt(0)) &&
                  !STOPWORDS.contains(word.toLowerCase) }.toSeq
    }
    def getNEs(textList: Seq[String]): Seq[Seq[String]] = {
        textList.map(getNEsSingle)
    }
    
    // Counts Named Entities
    def countNEs(result: Seq[Seq[String]]): Map[String, Int] = {
        result.flatten.foldLeft(Map.empty[String, Int]) {
         (count, word) => count + (word -> (count.getOrElse(word, 0) + 1)) }
    }
    
    // Sorts Named Entities
    def sortNEs(counts: Map[String, Int]): List[(String, Int)] = {
        counts.toList.sortBy(_._2)(Ordering[Int].reverse)
    } 
}

defined class NERModel

In [8]:
val model = new NERModel(STOPWORDS)

model: NERModel = ammonite.$sess.cmd6$Helper$NERModel@20f05553

### 2.2 Aplicar el "Modelo" a los datos

In [9]:
val result = model.getNEs(rssText)

result: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Chicago",
    "Red",
    "Stars",
    "SeatGeek",
    "Stadium",
    "Chicago",
    "Red",
    "Stars",
    "SeatGeek",
    "Stadium",
    "Bridgeview",
    "Saturday",
    "NJ/NY",
    "Gotham",
    "FC"
  ),
  ArrayBuffer(
    "Chicago",
    "Fire",
    "Inter",
    "Miami",
    "Luka",
    "Stojanovi\u0107",
    "Chicago",
    "Fire",
    "Inter",
    "Miami",
    "Saturday",
    "Soldier",
    "Field"
  ),
  ArrayBuffer(
    "Phil",
    "Mickelson",
    "PGA",
    "Championship",
    "Phil",
...

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [10]:
val counts = model.countNEs(result)
val sortedNEs = model.sortNEs(counts)

counts: Map[String, Int] = Map(
  "Much" -> 1,
  "Gleyber" -> 1,
  "Mercedes" -> 2,
  "Pederson" -> 2,
  "Parker" -> 2,
  "Bridgeview" -> 1,
  "John" -> 2,
  "Yerm\u00edn" -> 2,
  "Foster\u2019s" -> 1,
  "Canadiens" -> 2,
  "Column" -> 1,
  "Yankee" -> 2,
  "Sky" -> 4,
  "Mariners" -> 2,
  "Rod\u00f3n" -> 1,
  "Skys" -> 1,
  "Sunday\u2019s" -> 1,
  "Louis" -> 2,
  "Cease" -> 1,
  "FC" -> 1,
  "Foster" -> 1,
  "Olympic" -> 1,
  "Sox\u2019s" -> 1,
  "Rod\u00f3n\u2019s" -> 1,
  "Kyle" -> 1,
  "MVP" -> 1,
  "IOC" -> 1,
  "Mickelson" -> 3,
  "Matt" -> 2,
  "Maple" -> 1,
  "Side" -> 1,
  "Red" -> 2,
  "Tavares" -> 2,
  "What\u2019s" -> 1,
  "White" -> 11,
  "QB" -> 1,
  "Stadium" -> 5,
  "Underwood\u2019s" -> 1,
...
sortedNEs: List[(String, Int)] = List(
  ("Chicago", 29),
  ("Sox", 12),
  ("White", 11),
  ("Friday", 7),
  ("Cubs", 7),
  ("Stadium", 5),
  ("York", 5),
  ("New", 5),
  ("Sky", 4),
  ("Saturday", 4),
  ("Bears", 4),
  ("Yankees", 4),
  ("Mickelson", 3),
  ("PGA", 3),
  ("Anders